# 1. Import the libraries, load dataset, print shape of data, data description. (5 Marks)

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import os
import re, string, unicodedata
from textblob import TextBlob
import pandas as pd     
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
import nltk  

from nltk.corpus import stopwords, wordnet    # Stopwords, and wordnet corpus
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [2]:
data = pd.read_csv("tweets.csv")

In [3]:
print(data.shape)

(14640, 15)


In [4]:
data.columns.values

array(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'], dtype=object)

In [5]:
data.describe()

,tweet_id,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,1.464000e+04,14640.000000,10522.000000,14640.000000
mean,5.692184e+17,0.900169,0.638298,0.082650
std,7.791112e+14,0.162830,0.330440,0.745778
min,5.675883e+17,0.335000,0.000000,0.000000
25%,5.685592e+17,0.692300,0.360600,0.000000
50%,5.694779e+17,1.000000,0.670600,0.000000
75%,5.698905e+17,1.000000,1.000000,0.000000
max,5.703106e+17,1.000000,1.000000,44.000000


In [6]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
tweet_id,14640.0,5.692184e+17,7.791112e+14,5.675883e+17,5.685592e+17,5.694779e+17,5.698905e+17,5.703106e+17
airline_sentiment_confidence,14640.0,9.001689e-01,1.628300e-01,3.350000e-01,6.923000e-01,1.000000e+00,1.000000e+00,1.000000e+00
negativereason_confidence,10522.0,6.382983e-01,3.304398e-01,0.000000e+00,3.606000e-01,6.706000e-01,1.000000e+00,1.000000e+00
retweet_count,14640.0,8.265027e-02,7.457782e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.400000e+01


In [7]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


# 2. Understand of data-columns: (5 Marks)
a. Drop all other columns except “text” and “airline_sentiment”.
b. Check the shape of data.
c. Print first 5 rows of data.

In [8]:
# Instead of dropping columns, I am going to create a new DF with just two columns 

In [9]:
tweetDf = data[['text','airline_sentiment']]

In [10]:
tweetDf.shape

(14640, 2)

In [11]:
tweetDf.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


# 3. Text pre-processing: Data preparation. (20 Marks)

a. Html tag removal.
b. Tokenization.
c. Remove the numbers.
d. Removal of Special Characters and Punctuations.
e. Conversion to lowercase.
f. Lemmatize or stemming.
g. Join the words in the list to convert back to text string in the dataframe. (So that each row contains the data in text format.)
h. Print first 5 rows of data after pre-processing.

In [12]:
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []                            # Create empty list to store pre-processed words.
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)                # Append processed words to new list.
    return stems

In [13]:
def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []                           # Create empty list to store pre-processed words.
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)              # Append processed words to new list.
    return lemmas

In [14]:
def cleanData( textValue ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    
    
    # 1. Remove HTML
    review_text = BeautifulSoup(textValue).get_text() 
    #

    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    
    # 3. Tokenization
    review_text = nltk.word_tokenize(letters_only)
    #
    
    # 4. Remove non ascii
    ascii_word = unicodedata.normalize('NFKD', letters_only).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    #
    
    # 5. Remove punctuation
    new_word = re.sub(r'[^\w\s]', '', ascii_word)
    #
    
    # 6. Convert to lower case
    words = letters_only.lower().split()                             
    #
    
    # 7. Remove punctuation - Stemming 
    stems = stem_words(words)
    #   

    # 8. Join the words back into one string separated by space, and return the result.
    return( " ".join( words ))  

In [15]:
for i, row in tweetDf.iterrows():
    words = tweetDf.at[i, 'text']
    cleanedWords = cleanData(words)
    tweetDf.at[i,"cleanText"] = cleanedWords

C:\Users\dpandharkar\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\dpandharkar\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [16]:
tweetDf.head()

,text,airline_sentiment,cleanText
0,@VirginAmerica What @dhepburn said.,neutral,virginamerica what dhepburn said
1,@VirginAmerica plus you've added commercials t...,positive,virginamerica plus you ve added commercials to...
2,@VirginAmerica I didn't today... Must mean I n...,neutral,virginamerica i didn t today must mean i need ...
3,@VirginAmerica it's really aggressive to blast...,negative,virginamerica it s really aggressive to blast ...
4,@VirginAmerica and it's a really big bad thing...,negative,virginamerica and it s a really big bad thing ...


In [17]:
tweetDf['airline_sentiment'].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [18]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(tweetDf.airline_sentiment)
tweetDf['categorical_label'] = le.transform(tweetDf.airline_sentiment)

C:\Users\dpandharkar\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [19]:
tweetDf.head()

,text,airline_sentiment,cleanText,categorical_label
0,@VirginAmerica What @dhepburn said.,neutral,virginamerica what dhepburn said,1
1,@VirginAmerica plus you've added commercials t...,positive,virginamerica plus you ve added commercials to...,2
2,@VirginAmerica I didn't today... Must mean I n...,neutral,virginamerica i didn t today must mean i need ...,1
3,@VirginAmerica it's really aggressive to blast...,negative,virginamerica it s really aggressive to blast ...,0
4,@VirginAmerica and it's a really big bad thing...,negative,virginamerica and it s a really big bad thing ...,0


In [20]:
tweetDf.shape

(14640, 4)

# 4 Vectorization: (10 Marks)

# # 4.1 CountVectorizer

In [21]:
# Vectorization (Convert text data to numbers).
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=1000)                # Keep only 1000 features as number of features will increase the processing time.
countVectorFeatures = vectorizer.fit_transform(tweetDf['cleanText'])

countVectorFeatures = countVectorFeatures.toarray()                        # Convert the data features to array.

In [22]:
countVectorFeatures.shape

(14640, 1000)

# # 4.2 TfidfVectorizer

In [23]:
# Using TfidfVectorizer to convert text data to numbers.

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000)
TfidfFeatures = vectorizer.fit_transform(tweetDf['cleanText'])

TfidfFeatures = TfidfFeatures.toarray()



In [24]:
TfidfFeatures.shape

(14640, 1000)

# 5.  Fit and evaluate model using both type of vectorization. (6+6 Marks)

# Model using CountVectorizer

In [25]:
labels = tweetDf['categorical_label']
labels = labels.astype('int')

In [26]:
# Split data into training and testing set.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(countVectorFeatures, labels, test_size=0.3, random_state=42)

In [27]:
# Using Random Forest to build model for the classification of reviews.
# Also calculating the cross validation score.
import numpy as np  
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

forest = RandomForestClassifier(n_estimators=10, n_jobs=4)

forest = forest.fit(X_train, y_train)

print(forest)

print(np.mean(cross_val_score(forest, countVectorFeatures, labels, cv=10)))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=4,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)
0.7081284153005465


In [28]:
# Predict the result for test data using the model built above.

result = forest.predict(X_test)

In [29]:
# Print and plot Confusion matirx to get an idea of how the distribution of the prediction is, among all the classes.

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(y_test, result)

print(conf_mat)


[[2589  166   59]
 [ 463  371   50]
 [ 257   99  338]]


In [30]:
from sklearn.ensemble import GradientBoostingClassifier
gbk = GradientBoostingClassifier(random_state=42, n_estimators=100,min_samples_split=100)
gbk.fit(X_train, y_train)
gbk_predict = gbk.predict(X_test)

In [31]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,gbk_predict ))

0.7577413479052824


# Model using TfidfVectorizer

In [32]:
# Split data into training and testing set.

from sklearn.model_selection import train_test_split

X_train1, X_test1, y_train1, y_test1 = train_test_split(TfidfFeatures, labels, test_size=0.3, random_state=42)

In [33]:
# Using Random Forest to build model for the classification of reviews.
# Also calculating the cross validation score.
import numpy as np  
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

forest1 = RandomForestClassifier(n_estimators=10, n_jobs=4)

forest1 = forest.fit(X_train1, y_train1)

print(forest1)

print(np.mean(cross_val_score(forest1, TfidfFeatures, labels, cv=10)))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=4,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)
0.7126366120218579


In [34]:
# Predict the result for test data using the model built above.

result1 = forest1.predict(X_test1)

In [35]:
# Print and plot Confusion matirx to get an idea of how the distribution of the prediction is, among all the classes.

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

conf_mat1 = confusion_matrix(y_test1, result1)

print(conf_mat1)

[[2624  126   64]
 [ 485  334   65]
 [ 261   91  342]]


In [36]:
from sklearn.ensemble import GradientBoostingClassifier
gbk = GradientBoostingClassifier(random_state=42, n_estimators=100,min_samples_split=100)
gbk.fit(X_train1, y_train1)
gbk_predict = gbk.predict(X_test1)


In [37]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test1,gbk_predict ))

0.761384335154827


# 6. Summarize your understanding of the application of Various Pre-processing and Vectorization and performance of your model on this dataset. (8 Marks)

# Pre Processing :

Pre processing is required to bring the text into the form which can be analyzed and predicted.

Cleaning of text data is done to 
1. Remove unwanted information
2. Remove bias
3. Reduce volume to increase processing time
4. Convert into a form which is can be used by machine learning model

In above example, our task is to understand customer sentiments about airline using tweets. The sentiment can be positive, negative or neutral. 

We need to remove some unwanted information before collection of tweets can be analyzed by a machine learning model.  

Following pre processing steps are followed in above example.

1. Html tag removal - Because tweets data is collected using web scraping, it has HTML tags which do not add any value to meaning or understanding of sentiment.

2. Tokenization - It is a method in which sentences are converted into words.

3. Remove the numbers or Removal of Special Characters and Punctuations - remove unnecessary letters or words

4. Conversion to lowercase - The tokenized words are converted into lower case format. (India -> india). If they are not converted into lowercase then these both will constitute as non-identical words in the vector space model.

5. Stemming - It is the process in which the words are converted to its base from. Instead of Lemmatixing, stemming is used as pre processing step in above example.


# Vectorization:

CountVectorizer counts the word frequencies.  

With the TFIDFVectorizer, the value increases proportionally to count, but is inversely proportional to frequency of the word in the corpus; that is the inverse document frequency (IDF) part. In TfidfVectorizer, we consider overall document weightage of a word.

In CountVectorizer, we only count the number of times a word appears in the document which results in biasing in favour of most frequent words. This ends up in reduced weight of less frequent words. To avoid this, TFIDFVectorizer is used.


# Model Creation and Prediction:

With RandomForestClassifier, the accuracy is almost same for both vectorization methods.

From above example, the accuracy is more when TfidfVectorizer and GradientBoostingClassifier is used. The accuracy score for GradientBoostingClassifier with TfidfVectorizer is 0.76.

The accuracy can be improved further using different pre processing steps and different model parameters.

We have restricted the number of features to 1000 for vectorization. I tried increasing it to 5000 but it did not make any difference to accuracy but increased processing time a lot for model predict function. 

